# CrewAI

How to use:
- Run "Setup"
- Jump to one of the chapters and execute the cells, all chapters are independent from each other

# TOC

1. [CrewAI: Tool Tests](#CrewAI:-Tool-Tests)
1. [Web Researcher](#1.-Web-Researcher)

# Notes

## Tools

- ServerDevTool (Google) works better than DuckDuckGoSearchRun (DuckDuckGo). Eg. it provides the dates of the found articles, which DuckDuckGoSearchRun does not do

# Setup

In [ ]:
#!uv pip install jupyterlab==4.2.1 pyautogen==0.2.27 crewai==0.30.11 crewai-tools==0.2.6 duckduckgo-search==6.1.4 agentops==0.2.0

In [ ]:
#uv pip freeze > requirements2.txt
#uv pip install -r requirements.txt

In [48]:
# Warning control
import warnings
#warnings.filterwarnings("ignore")
#warnings.simplefilter("ignore", category=Warning)

In [2]:
import json
import os
from getpass import getpass
import psutil
import openai
IN_NOTEBOOK = any(["jupyter-lab" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string):  ········


In [39]:
from os import environ
if environ.get('OPENAI_API_KEY') is None:
    print('Environment variable not found. Setting values...')
    os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential'] 
    os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
    os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']
    os.environ["SERPER_API_KEY"] = CREDS['serper_dev']['key']['credential']
    #openai.api_key = CREDS['OpenAI']['v2']['credential'] 

In [ ]:
# Enable Langchain Tracing
#os.environ["LANGCHAIN_API_KEY"] = CREDS['langsmith']['key']['credential']
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
# Enable Agentop Tracing
#os.environ["AGENTOPS_API_KEY"] = CREDS['agentops']['key']['credential']
#import agentops
#agentops.init()

# Get list of available OpenAI models

In [4]:
client = openai.OpenAI()
model_list = client.models.list()
for model in model_list:
  print(model.id)

dall-e-3
gpt-4-1106-preview
whisper-1
davinci-002
babbage-002
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-3.5-turbo-1106
gpt-4o-2024-05-13
gpt-4o
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
text-embedding-3-small
gpt-4-turbo-2024-04-09
tts-1
gpt-4-turbo
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-4-turbo-preview
tts-1-1106
text-embedding-3-large
gpt-3.5-turbo-0613
gpt-4
text-embedding-ada-002
gpt-4-1106-vision-preview
gpt-3.5-turbo-16k-0613
gpt-4-0613
gpt-4-vision-preview


# CrewAI: Tool Tests: Web search, files

- Simple crew just for testing some tools
- The crew is created by a function `createCrew(mytools, myverbosity)` which makes it easier to check different tools with the same crew

In [40]:
from crewai import Agent, Task, Crew, Process

os.environ["OPENAI_MODEL_NAME"]= "gpt-3.5-turbo" # pr "gpt-4o" # or "gpt-4-1106-preview"

# file tools
from crewai_tools import DirectoryReadTool, FileReadTool
directory_read_tool = DirectoryReadTool(directory='./campaign-instructions')
file_read_tool = FileReadTool()

# file write: see https://github.com/joaomdmoura/crewAI-examples/blob/main/landing_page_generator/tools/file_tools.py

# online search tools
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun
search_tool_google = SerperDevTool()
search_tool_duckduckgo = DuckDuckGoSearchRun()
search_tool = search_tool_google

# Creating a crew, tools can be passed by argument
def createCrew(mytools, myverbosity):
    researcher = Agent(
      role='Senior Researcher',
      goal='Answer the following question: {topic}',
      verbose=True if myverbosity == True else False,
      memory=True,
      backstory=(
        "Driven by curiosity, you're at the forefront of"
        "information research, eager to explore and find knowledge."
      ),
      tools= mytools
      #allow_delegation=True
    )

    answer = Task(
      description=(
        "Answer the question '{topic}'."
        "Answer in a short and precise way, do not add anything else." # Use bulletpoints to structure your answer. "
        "Write your answer to a file."
      ),
      expected_output="The answer to the question '{topic}'", #,formatted as markdown.
      #tools=[search_tool],
      agent=researcher,
      async_execution=False,
      output_file='agent-output.txt'  # This is necessary to create the output as file
    )

    # Forming the tech-focused crew with some enhanced configurations
    crew = Crew(
      agents=[researcher],
      tasks=[answer],
      process=Process.sequential,  # Optional: Sequential task execution is default
      memory=True,
      cache=False,
      max_rpm=100,
      share_crew=True,
      verbose=2 if myverbosity == True else 0,
    )
    return crew


In [50]:
crew = createCrew([directory_read_tool, file_read_tool], False)
result = crew.kickoff(inputs={'topic': 'What keypoints should be adressed for enterprise solutions'}) 
print(result)

2024-06-03 14:37:35,511 - 8447457984 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 

Fail to read the file ./campaign-instructions/keypoints_enterprise_solutions.txt. Error: [Errno 2] No such file or directory: './campaign-instructions/keypoints_enterprise_solutions.txt'

 

File paths: 
-./campaign-instructions/enterprise_solutions_framework.md
- ./campaign-instructions/tech_startups_outreach.md
- ./campaign-instructions/small_business_engagement.md
- ./campaign-instructions/.ipynb_checkpoints/enterprise_solutions_framework-checkpoint.md

The answer to the question 'What keypoints should be adressed for enterprise solutions' is located in the file ./campaign-instructions/enterprise_solutions_framework.md.


In [51]:
crew = createCrew([], False)
result = crew.kickoff(inputs={'topic': 'What is 10+20?'}) 
print(result)

2024-06-03 14:38:13,901 - 8447457984 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


The answer to the question 'What is 10+20?' is 30.


In [29]:
from IPython.display import Markdown,display
display(Markdown(result))

- **Strategic Partnership:** Convey the value of a long-term partnership.
- **Integration:** Highlight the ease of integration with their current systems.
- **Innovation:** Showcase how your solutions contribute to driving innovation within their organization.

# 1. Web Researcher 

- searches Google and returns Top 5 results report
- adopted from example below

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun

search_tool_google = SerperDevTool()
search_tool_duckduckgo = DuckDuckGoSearchRun()

search_tool = search_tool_google

os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo" # or "gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent
researcher = Agent(
  role='Senior Researcher',
  goal='Find most relevant sources in the web {topic}',
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "information research, eager to explore and find knowledge."
  ),
  tools=[search_tool],
  allow_delegation=False,
	verbose=True
)

# Creating a writer agent with 
writer = Agent(
  role='Writer',
  goal='Writing precise reports about {topic}',
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "reports and summaries on an expert level."
  ),
  allow_delegation=False,
  verbose=True
)

In [ ]:
# Research task
research_task = Task(
  description=(
    "Search the web for the most relevant articles about {topic} in 2024."
    "Focus on sources like scientific papers, newspapers and expert blogs."
    "Your final report should list the 5 most relevant articles including URL, date, headline and a short summary."
  ),
  expected_output='A comprehensive list of search results.',
  tools=[search_tool],
  agent=researcher
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful report on {topic}."
    "Take the given articles and write for each of them one chapter. Include the source URL and date."
  ),
  expected_output='A 4 report on {topic}, listing the most relevant sources, formatted as markdown.',
  #tools=[search_tool_google],
  agent=writer
)

In [ ]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [ ]:
# Starting the task execution process with enhanced feedback
# result = crew.kickoff(inputs={'topic': 'will donald trumps chances of winning the presidential election diminish after his conviction in court?'})
# result = crew.kickoff(inputs={'topic': 'Will Apples Siri get smarter in 2024?'}) 
result = crew.kickoff(inputs={'topic': 'Will Apples Siri get smarter in 2024? Answer in German language'}) 
print(result)

In [ ]:
from IPython.display import Markdown,display
display(Markdown(result))

# 2. Blog post writer: Researcher, Writer

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

os.environ["OPENAI_MODEL_NAME"]="gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

In [ ]:
# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [ ]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'}) # in healthcare # in software development
print(result)

In [ ]:
print(result)

In [ ]:
print(result)

# 3. Blog post writer: Planner, Writer, Editor

Blog post writer Multi-Agent System using Crewai and Ollama

Source: https://medium.com/the-ai-forum/create-a-blog-writer-multi-agent-system-using-crewai-and-ollama-f47654a5e1cd

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model = "crewai-llama3",
    base_url = "http://localhost:11434/v1")

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} in 'https://medium.com/'."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "You have to prepare a detailed "
              "outline and the relevant topics and sub-topics that has to be a part of the"
              "blogpost."
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    #llm=llm,
    allow_delegation=False,
 verbose=True
)
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic} in 'https://medium.com/'. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    #llm=llm,
    verbose=True
)
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization 'https://medium.com/'. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    #llm=llm,
    allow_delegation=False,
    verbose=True
)

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown,display
display(Markdown(result))

# 4. Poem Writer: Manager, Researcher, Writer

Source: https://medium.com/thedeephub/crafting-custom-love-poems-with-crewai-agents-5ee3051b4414

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain.tools import tool
from crewai.tasks.task_output import TaskOutput
from langchain_openai import ChatOpenAI

In [ ]:
# Define the topic 
# In this case, since we want a poem for our loved one, the topic should be things that they like and that you want to include in the poem
topic = "Taylor Swift and long walks on the beach at sunset"

In [ ]:
# Define the search tool 
search_tool = DuckDuckGoSearchRun()

# Function that is run after the task is completed
def callback_function(output: TaskOutput):
    ## What to do after the task is completed (i.e. send an sms with a love poem)
    print(f"""
        Task completed with the following output:
        Task: {output.description}
        Output: {output.result}
    """)

# Custom tool - read webpage
class ContentTools:
    @tool("Read webpage content")
    def read_content(url: str) -> str:
        """Read the content of a webpage"""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text()
        return text_content[:5000]

In [ ]:
# First agent: the love poem manager
manager = Agent(
    role = "Love Poem Manager",
    goal = "Manage the process of creating a love poem for the partner of the client",
    verbose = True,
    backstory = "With a mind for romance, I want to deliver the best custom love poem to my client to give to their partner for Valentines Day",
    allow_delegation = True,
    max_iter = 10,
    max_rpm = 20,
)
# Second agent: the love poem searcher
researcher = Agent(
    role = "Love Poem Searcher",
    goal = f"Find a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a curious love poem searcher, and you want to 
    find the best and most custom love poem for your client to give to their partner, 
    therefore you need some details about the things that the partner really likes! You will search for these details.""",
)
# Third agent: the love poem writer
writer = Agent(
    role = "Love Poem Writer",
    goal = f"Write a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a creative love poem writer, and you want to 
    write the best and most custom love poem for your client to give to their partner. 
    With a mind for romance and a deep love for poetry, you find the perfect poetry style 
    for your client and create a wonderful poem that conveys love, respect, kindness, and admiration""",
)

In [ ]:
# Task 1: List ideas for a love poem
list_ideas = Task(
    description = "List of 5 cute ideas for a love poem, including possible poetry styles",
    expected_output = "Bullet point of 5 cute ideas for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

# Task 2: List important facts related to topic
list_facts = Task(
    description = f"List of 5 important facts specifically for someone that likes {topic}",
    expected_output = "Bullet point of 5 important facts related to the given topic for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

# Task 3: Write the love poem
write_poem = Task(
    description = f"Write a poem that would make Romeo and Juliet blush. The poem should be custom and tailored to the client's partner, who really likes {topic}",
    expected_output = "A lovely poem in a style that the client would like",
    tools=[search_tool],
    agent = writer,
    context=[list_ideas, list_facts],
    callback=callback_function,
)

# Task 4: Coordinate the love poem
manager_task = Task(
    description=f"""Oversee the creation of a great love poem for the client, 
    a lovebird who's looking for a custom poem for their partner, who really likes {topic}. 
    Ensure that the poem is custom and tailored to the client's partner and in a style that 
    is appropriate and conveys love, respect, kindness, and admiration""",
    expected_output=f"A lovely poem in a style that the client would like that incoporates the ideas and facts related to the {topic}",
    agent=manager,
)

In [ ]:
crew = Crew(
    agents = [manager, researcher, writer],
    tasks = [list_ideas, list_facts, write_poem, manager_task],
    process = Process.hierarchical,
    manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),
)
result = crew.kickoff()

In [ ]:
print(result)

# 5. Outreach campaign manager

- Source: https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/9/tools-for-a-customer-outreach-campaign-(code)

In [7]:
from crewai import Agent, Task, Crew, Process
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [8]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

# tools
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool, BaseTool

directory_read_tool = DirectoryReadTool(directory='./campaign-instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"
sentiment_analysis_tool = SentimentAnalysisTool()

# tasks
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

# crew
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [11]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a detailed profile and identify potential areas where our solutions can provide value.

Action:
Search the internet

Action Input:
{"search_query": "DeepLearningAI company profile"}
 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplearning-ai
Sn

In [12]:
from IPython.display import Markdown
Markdown(result)

Email Draft to Andrew Ng:

Subject: Elevating DeepLearningAI's Impact: A Partnership Proposal from CrewAI

Dear Andrew Ng,

I hope this email finds you well. I am reaching out from CrewAI, where we specialize in providing tailored solutions to enhance online learning platforms and create personalized learning experiences. Your pioneering work at DeepLearningAI has not gone unnoticed, and we admire the significant strides your company has made in the AI education space.

Our team at CrewAI is passionate about collaborating with innovative leaders like yourself to drive forward-thinking initiatives in the field of AI and machine learning. We have been following DeepLearningAI's recent activities closely, including the updates to your courses, and we are impressed by the commitment to excellence and continuous improvement.

As you continue to expand the reach and impact of DeepLearningAI, we believe that our solutions can complement your existing offerings and support your growth in the AI education sector. We see a potential opportunity for collaboration on developing advanced AI courses or projects that align with DeepLearningAI's mission of making world-class AI education accessible to a global audience.

At CrewAI, we prioritize personalized engagement strategies focused on collaboration and innovation. We are confident that by combining our expertise with your vision, we can create a successful partnership that elevates DeepLearningAI's impact and unlocks new possibilities for the future of AI education.

I would love the opportunity to discuss how CrewAI can support DeepLearningAI's goals and contribute to the continued success of your organization. Please let me know a convenient time for a meeting or call to explore potential synergies and opportunities for collaboration.

Thank you for considering this partnership proposal. We look forward to the possibility of working together to drive innovation and excellence in the AI education landscape.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Please note that the email draft has been tailored to resonate with DeepLearningAI's recent achievements, company culture, and Andrew Ng's leadership. It emphasizes the potential for collaboration and highlights how CrewAI's solutions can support DeepLearningAI's goals and mission.